In [ ]:
from glotaran.io import load_parameters

parameters = load_parameters("models/parameters_equal_area_penalties.yml")

In [ ]:
from glotaran.utils.json_schema import create_model_scheme_json_schema

create_model_scheme_json_schema("models/schema.json", parameters)

### Analysis without equal area penalties

Example of a two step optimization
- First do a (partial) optimization with first scheme
- Extract the optimized schemed and adjust its settings
- Optimize again with the second scheme

In [ ]:
from glotaran.io import load_scheme

scheme = load_scheme("models/scheme.yml")

In [ ]:
from glotaran.io import load_dataset

scheme.load_data({"dataset1": load_dataset("data/data.ascii")})

In [ ]:
result_no_penalties_first_run = scheme.optimize(
    parameters=parameters, maximum_number_function_evaluations=7
)

#### Saving results and creating plots

In [ ]:
from pyglotaran_extras.io import setup_case_study

results_folder, _ = setup_case_study(output_folder_name="pyglotaran_examples_results_staging")
result_no_penalties_first_run.save(results_folder / "no_penalties_first_run", allow_overwrite=True)

In [ ]:
result_no_penalties = scheme.optimize(
    parameters=result_no_penalties_first_run.parameters_optimized
)
result_no_penalties.save(results_folder / "no_penalties", allow_overwrite=True)

In [ ]:
from pyglotaran_extras import plot_overview
from pyglotaran_extras.compat import convert

plot_overview(convert(result_no_penalties_first_run.data["dataset1"]))
plot_overview(convert(result_no_penalties.data["dataset1"]));

### Analysis using equal area penalties

This time we use equal parameter in our model, allowing us to estimate the relative amplitude between the two species.

In [ ]:
parameters_equal_area = load_parameters("models/parameters_equal_area_penalties.yml")

scheme_equal_area = load_scheme("models/scheme_equal_area_penalties.yml")
scheme_equal_area.load_data({"dataset1": load_dataset("data/data.ascii")})

result_equal_area_first_run = scheme_equal_area.optimize(parameters=parameters_equal_area)

In [ ]:
result_equal_area_first_run.save(results_folder / "with_penalties_first_run", allow_overwrite=True)

In [ ]:
result_equal_area = scheme_equal_area.optimize(
    parameters=result_equal_area_first_run.parameters_optimized
)
result_equal_area.save(results_folder / "with_penalties", allow_overwrite=True)

In [ ]:
plot_overview(convert(result_equal_area_first_run.data["dataset1"]))
plot_overview(convert(result_equal_area.data["dataset1"]));

In [ ]:
from glotaran.io import save_dataset

for data_path in results_folder.rglob("*.nc"):
    save_dataset(
        convert(load_dataset(data_path)), results_folder / data_path.name, allow_overwrite=True
    )